# 1. Import and Install Dependencies

## 1.2 Load Dependencies

In [ ]:

import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import pandas as pd
import librosa
import numpy as np
import math
from tqdm import tqdm
from joblib import Parallel, delayed  
import multiprocessing



In [ ]:
# Parameters

clip_duration_sec = 5
frame_length=1024
frame_step=128



# 2. Build Data Loading Function

## 2.1 Define Paths to Files

In [ ]:
df = pd.read_csv("/mnt/disks/birdclef-2022/all_data_df.csv")
# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
# Proc
different_birds = list(df["files"])
df["bird_type"] = df.apply(lambda x: x["files"].split("/")[-2], axis=1)
df

In [ ]:
df["files"][0]

In [ ]:
df["files"] = df.apply(lambda x: x["files"].replace(
    "/Users/viktorcikojevic/coding/identify_bird_calls/birdclef-2022/train_audio", 
    "/mnt/disks/birdclef-2022/birdclef-2022-data/train_audio"
), axis=1)

In [ ]:
# Example
df_rinduc = df[df["bird_type"] == "rinduc"]
df_rinduc = df_rinduc.reset_index(drop=True)
df_rinduc

In [ ]:

file_name = df_rinduc["files"][4]
print(file_name)
wav, sample_rate = librosa.load(file_name, duration=5, sr=16000 )
plt.plot(wav)

In [ ]:
def get_spectrogram(wav, clip_duration_sec=5):
    # 16000 data = 1 seconds
    num_data = int(16000 * clip_duration_sec)
    wav = wav[:num_data]
    zero_padding = tf.zeros([num_data] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=frame_length, frame_step=frame_step)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram / tf.reduce_max(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram.numpy()

In [ ]:
n_grid = 10

def moving_average(x, w):
    arr =  np.convolve(x, np.ones(w), 'valid') / w
    return arr / np.max(arr)

df_rinduc = df[df["bird_type"] == "sora"]
df_rinduc = df_rinduc.reset_index(drop=True)

fig, axs = plt.subplots(n_grid, 3, figsize=(15,30))
for i in range(n_grid):
    indx = i + 10
    file_name = df_rinduc["files"][indx]
    print(file_name)
    wav, sample_rate = librosa.load(file_name, duration=5, sr=16000, offset=0 )
    wav = wav - np.average(wav)
    
    axs[i,0].plot(wav)
    spectrogram = np.abs(np.fft.fftn(wav))
    spectrogram *= 1. / np.max(spectrogram)
    spectrogram = spectrogram[0:int(len(spectrogram)/2)]
    
    axs[i, 1].plot(spectrogram)
    
    
    spectrogram = moving_average(spectrogram, 400)
    axs[i, 2].plot(spectrogram[::100])
    x_shape = np.shape(spectrogram)[0]
    x = np.arange(1, x_shape+1)
    spectrogram = spectrogram * np.exp(-(2000 / x)**5)
    spectrogram = spectrogram[::100]
    axs[i, 2].plot(spectrogram)
    print(np.shape(spectrogram))
    
    # wav, sample_rate = librosa.load(file_name, duration=5, sr=16000, offset=0 )
    # wav = wav / np.average(wav)
    # # spectrogram = np.abs(np.fft.fftn(wav))
    # # spectrogram *= 1. / np.max(spectrogram)
    # spectrogram = spectrogram[0:int(len(spectrogram)/2)]
    # x_shape = np.shape(spectrogram)[0]
    # x = np.arange(1, x_shape+1)
    # spectrogram = spectrogram * np.exp(-(2000 / x)**5)
    # spectrogram = np.append(spectrogram, np.flip(spectrogram))
    # wav = np.fft.ifftn(spectrogram)
    
    # axs[i,3].plot(wav)
    #spectrogram = get_spectrogram(wav)
    #axs[i,3].imshow(spectrogram, vmin=0, vmax=0.1)

In [ ]:
#np.shape(np.reshape(spectrogram, (10, 4000)))

In [ ]:
np.shape(spectrogram)

In [ ]:


n_grid = 3

df_rinduc = df[df["bird_type"] == "jabwar"]
df_rinduc = df_rinduc.reset_index(drop=True)

fig, axs = plt.subplots(n_grid, 2, figsize=(8,15))
for i in range(n_grid):
    file_name = df_rinduc["files"][i]
    print(file_name)
    wav, sample_rate = librosa.load(file_name, duration=5, sr=16000, offset=0 )
    axs[i,0].plot(wav)
    spectrogram = get_spectrogram(wav)
    axs[i,1].imshow(spectrogram, vmin=0, vmax=0.01)

In [ ]:
import math
# Loop through each audio file
# For a current file, calculate the spectrogram, and kill of the low frequncy components
# Save that numpy file to some folder. Remember the file path for the dataframe
# If the maximum height is below 0.6, classify this as "nocall". Otherwise, keep the label as before. 
# Save the labels in the csv file and upload to the kaggle. 
# implement the xgboost  for this !!

def load_feature_file(file_path, offset):
    # Load the wav file
    wav, sample_rate = librosa.load(file_path, duration=5, sr=16000, offset=offset)
    # Make average equal to zero to avoid sporious k=0 FFT contributions
    wav = wav - np.average(wav)
    # Take only 5 second intervals. If the file is less then 5 seconds, pad with zeros.
    num_data = int(16000 * 5)
    zero_padding = np.zeros(num_data - np.shape(wav)[0])
    wav = np.concatenate([zero_padding, wav])
    # Take the fft and normalize it
    spectrogram = np.abs(np.fft.fftn(wav))
    spectrogram *= 1. / np.max(spectrogram)
    spectrogram = spectrogram[0:int(len(spectrogram)/2)]
    # Calculate the moving average since the FFTs have a lot of noise
    spectrogram = moving_average(spectrogram, 400)
    x_shape = np.shape(spectrogram)[0]
    x = np.arange(1, x_shape+1)
    # Apply np.exp(-(2000 / x)**5) on the FFT, to remove noise around x=0 (low frequencies)
    spectrogram = spectrogram * np.exp(-(2000 / x)**5)
    # Take each 100 data point, since the FFTs have a lot of data
    spectrogram = spectrogram[::100]
    return spectrogram

files_new = []
bird_type_new_label = []
root_folder = "/mnt/disks/birdclef-2022/clean_ffts"

n_procs = 16

def process_file(i_offset, file_path, bird_type):
    offset = i_offset * 5
    feature = load_feature_file(file_path, offset)
    new_label = bird_type
    if np.max(feature) < 0.6:
        new_label = "nocall"
    
    bird_folder = os.path.join(root_folder, new_label)
    
    file_name = file_path.split("/")[-1].split(".")[0] + f"_{i_offset}"
    if new_label == "nocall":
        file_path.replace(bird_type, "nocall")
    file_path_new = os.path.join(bird_folder, file_name)
    np.save(file_path_new, feature)


bird_types = list(set(list(df["bird_type"])))
indx = 0
for bird_type in bird_types:
    print(f"Analyzing bird {bird_type}")
    df_bird = df[df["bird_type"] == bird_type]

    bird_folder = os.path.join(root_folder, bird_type)
    if os.path.exists(bird_folder) is False:
        os.mkdir(bird_folder)
    for file_path, duration_sec in zip(list(df_bird["files"]), list(df_bird["duration_sec"])):
        n_procs = multiprocessing.cpu_count()
        res = Parallel(n_jobs=n_procs)(delayed(process_file)(
                                            i_offset,
                                            file_path,
                                            bird_type
                                            ) for i_offset in range(int(duration_sec / 5)))   
                                

In [ ]:
df_with_silent = pd.DataFrame({
    "file_name" : file_path_new,
    "bird_type" : bird_type_new_label,
                               })
df_with_silent

In [ ]:
df[df["bird_type"] == "fragul"]